In [58]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [59]:
filename = "test_with_del__2_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

66432

In [60]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

493307

In [61]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1706536001
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 3321502


b'chr22\t1350\tA\tG\t-\nchr22\t1547\tG\tK\t+\nchr22\t1586\tC\t-\t-\nchr22\t3222\tA\tM\t+\nchr22\t5653\tG\tA\t-\nchr22\t11841\tA\tW\t+\nchr22\t13845\tC\tS\t+\nchr22\t14727\tT\tA\t-\nchr22\t16373\tG\tT\t-\nchr22\t16767\tG\tK\t+\nchr22\t18977\tC\tY\t+\nchr22\t19500\tA\tR\t+\nchr22\t20354\tC\tM\t+\nchr22\t20741\tA\tM\t+\nchr22\t23520\tC\tT\t-\nchr22\t24308\tA\tR\t+\nchr22\t29853\tT\t-\t-\nchr22\t30012\tA\tW\t+\nchr22\t31293\tT\tK\t+\nchr22\t31821\tT\tY\t+\nchr22\t32061\tG\tK\t+\nchr22\t32184\tT\tW\t+\nchr22\t33872\tT\tY\t+\nchr22\t35370\tC\tS\t+\nchr22\t37084\tC\tM\t+\nchr22\t37595\tG\tK\t+\nchr22\t38361\tC\tY\t+\nchr22\t38791\tC\tM\t+\nchr22\t38857\tC\tA\t-\nchr22\t40236\tA\tR\t+\nchr22\t40864\tG\tR\t+\nchr22\t41578\t-\tG\t-\nchr22\t41818\t-\tT\t-\nchr22\t42118\tG\tR\t+\nchr22\t42246\tC\tS\t+\nchr22\t44171\tT\tC\t-\nchr22\t46076\t-\tTG\t+\nchr22\t47101\tC\tM\t+\nchr22\t47690\t-\tGT\t+\nchr22\t49604\tT\tW\t+\nchr22\t50693\tA\tW\t+\nchr22\t54088\tA\tW\t+\nchr22\t54482\tA\tT\t-\nchr22\t54538

In [62]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [63]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

In [64]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [65]:
app.rd(["biasOut_sorted.bam"])

In [66]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [67]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [68]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4865	7.157220e-06	1.303840e-58	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.3981	0.000000e+00	5.586096e-269	0.000000e+00	9.546350e-240	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.1344	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4709600	163200	1.6661	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4709701-4713300	3600	1.7015	0.000000e+00	1.060163e-148	2.589803e-09	9.615301e-151	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4302	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854500	6200	1.6775	0.000000e+00	2.946859e-242	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047700	8300	1.9571	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424001-9461600	37600	1.4873	0.000000e+00	0.000000e+00	0.000000

In [69]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [70]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4865295077644867,0.0
duplication,chr22,1306101,1325100,19000,1.3981176030546358,0.0
duplication,chr22,2926201,2963000,36800,3.134430285010776,0.0
duplication,chr22,4546401,4709600,163200,1.6660880470430022,0.0
duplication,chr22,4709701,4713300,3600,1.7015080674277123,0.0
duplication,chr22,6655001,6661800,6800,2.43017254193035,0.0
duplication,chr22,6848301,6854500,6200,1.6774730826942141,0.0
duplication,chr22,8039401,8047700,8300,1.957071478717659,0.0
duplication,chr22,9424001,9461600,37600,1.4872890878282967,0.0
duplication,chr22,13230601,13395900,165300,1.4081980440343234,0.0
duplication,chr22,13448901,13453700,4800,1.3217685001778146,0.0
duplication,chr22,13480901,13512800,31900,4.240124724456376,0.0
duplication,chr22,15242601,15246400,3800,3.019587518905921,0.0
duplication,chr22,18662501,18664600,2100,1.457153444143763,0.0
duplication,c

In [71]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3882550089853372,0.0
duplication,chr22,2926001,2963000,37000,3.1031615104105037,0.0
duplication,chr22,4546001,4713000,167000,1.6550537811978285,0.0
duplication,chr22,6655001,6662000,7000,2.3788453210123626,0.0
duplication,chr22,8040001,8048000,8000,1.9361232034196818,0.0
duplication,chr22,9424001,9462000,38000,1.4732843005448732,0.0
duplication,chr22,13231001,13396000,165000,1.3986768122596749,0.0
duplication,chr22,13481001,13513000,32000,4.199969882265249,0.0
duplication,chr22,18705001,18881000,176000,2.7359868646595262,0.00014096184817631997
duplication,chr22,19259001,19268000,9000,3.2553514916082222,0.0
duplication,chr22,21248001,21381000,133000,1.4683270614779014,0.00013540655819096838
duplication,chr22,21533001,21548000,15000,1.2921666346336047,0.0026642496154691275
duplication,chr22,21582001,21624000,42000,1.5595989132023331,0.00

In [72]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.6465874092646493,0.0
duplication,chr22,6520001,7010000,490000,1.0752047989610782,0.0
duplication,chr22,13230001,13350000,120000,1.3921044314878472,0.0
duplication,chr22,18710001,18880000,170000,2.7142231458971695,0.0001454828713965783
duplication,chr22,21230001,21380000,150000,1.3799275778437186,0.00013785412817726071
duplication,chr22,25150001,25310000,160000,1.2683518363416335,0.0001291900307054689
duplication,chr22,28470001,28600000,130000,3.0764527979690817,4.274676951631341e-05
duplication,chr22,30800001,30980000,180000,2.8310825107015574,2.5984707999342388e-05
duplication,chr22,31150001,31360000,210000,1.4063901233230571,4.9660622850626166e-05
duplication,chr22,31870001,32250000,380000,1.2643727908796112,9.291413572432376e-05
duplication,chr22,35940001,36200000,260000,2.180768961036526,4.206224651654495e-05
duplication,chr22,3

In [73]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,45700001,51400000,5700000,1.589395432091682,7.262256031583978e-05


In [74]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [75]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__2_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [76]:
del app